<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/all_seeds_sorted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz

root_dir = './'
torchvision.datasets.MNIST(root=root_dir,download=True)

--2021-04-08 21:27:26--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-04-08 21:27:26--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.2’

MNIST.tar.gz.2          [            <=>     ]  33.20M  14.4MB/s    in 2.3s    

2021-04-08 21:27:29 (14.4 MB/s) - ‘MNIST.tar.gz.2’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
mnist_trainset = torchvision.datasets.MNIST(root=root_dir, train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = torchvision.datasets.MNIST(root=root_dir, 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

In [5]:
new_mnist_trainset =  [ [[],[]] for i in range(10)]
# new_mnist_testset  =  [ [[],[]] for i in range(10)]

for i in range(60000):
    for j in range(10):
        if mnist_trainset[i][1] == j:
            # image 
            new_mnist_trainset[j][0].append(mnist_trainset[i][0])  
            # label
            new_mnist_trainset[j][1].append(mnist_trainset[i][1])

# for i in range(10000):
#     for j in range(10):
#         if mnist_testset[i][1] == j:
#             # image 
#             new_mnist_testset[j][0].append(mnist_testset[i][0])  
#             # label
#             new_mnist_testset[j][1].append(mnist_testset[i][1])

image_trainset = list()
label_trainset = list()

# image_testset = list()
# label_testset = list()

for i in range(10):
    image_trainset.append(new_mnist_trainset[i][0])
    label_trainset.append(new_mnist_trainset[i][1])

# for i in range(10):
#     image_testset.append(new_mnist_testset[i][0])
#     label_testset.append(new_mnist_testset[i][1])

flattened_image_train = list()
flattened_label_train = list()

# flattened_image_test = list()
# flattened_label_test = list()

# flattening image 
for sublist in image_trainset:
    for val in sublist:
        flattened_image_train.append(val)

# flattening label
for sublist in label_trainset:
    for val in sublist:
        flattened_label_train.append(val)

# # flattening image 
# for sublist in image_testset:
#     for val in sublist:
#         flattened_image_test.append(val)

# # flattening label
# for sublist in label_testset:
#     for val in sublist:
#         flattened_label_test.append(val)

flattened_image_train = torch.stack(flattened_image_train)
flattened_label_train = torch.Tensor(flattened_label_train)
flattened_label_train = flattened_label_train.type(torch.LongTensor)

# flattened_image_test = torch.stack(flattened_image_test)
# flattened_label_test = torch.Tensor(flattened_label_test)
# flattened_label_test = flattened_label_test.type(torch.LongTensor)

train_dataset = TensorDataset(flattened_image_train, flattened_label_train)
train_dataloader = DataLoader(train_dataset, batch_size=50)

# test_dataset = TensorDataset(flattened_image_test, flattened_label_test)
test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

In [6]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [7]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [8]:
def selectivity(hidden_layer_each_neuron):
    __selectivity__ = list()
    # I will now try to find the average of each class for each neuron.
    # check out the next cell 
    avg_activations = [dict() for x in range(256)]
    for i, neuron in enumerate(hidden_layer_each_neuron):
        for k, v in neuron.items():
            # v is the list of activations for hidden layer's neuron k 
            avg_activations[i][k] = sum(v) / float(len(v))

    # generate 256 lists to get only values in avg_activations
    only_activation_vals = [list() for x in range(256)]

    # get only values from avg_activations
    for i, avg_activation in enumerate(avg_activations):
        for value in avg_activation.values():
            only_activation_vals[i].append(value)

    for activation_val in only_activation_vals:
        # find u_max 
        u_max = np.max(activation_val)

        # find u_minus_max 
        u_minus_max = (np.sum(activation_val) - u_max) / 9

        # find selectivity 
        selectivity = (u_max - u_minus_max) / (u_max + u_minus_max)

        # append selectivity value to selectivity
        __selectivity__.append(selectivity)

    avg_selectivity = np.average(__selectivity__)
    std_selectivity = np.std(__selectivity__)
                                 
    return avg_selectivity, std_selectivity

In [9]:
def sparsity_calculator(final_spareness):
    sparseness_list = list()
    for single_epoch_spareness in final_spareness:

        hidden_layer_activation_list = single_epoch_spareness
        hidden_layer_activation_list = torch.stack(hidden_layer_activation_list)
        layer_activations_list = torch.reshape(hidden_layer_activation_list, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return sparseness_list

In [10]:
no_epochs = 100
def selectivity_trainer(optimizer, model):

    criterion = torch.nn.CrossEntropyLoss()
    train_loss = list()
    test_loss  = list()
    test_acc   = list()

    final_spareness = list()
    
    final_selectivity_avg_list = list()
    final_selectivity_std_list = list()

    best_test_loss = 1

    for epoch in range(no_epochs):

        _hidden_layer_each_neuron_ = [{0:[], 1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[]} for x in range(256)]
        _hidden_layer_each_neuron_ = np.array(_hidden_layer_each_neuron_)

        hidden_layer_activation_list = list()

        total_train_loss = 0
        total_test_loss = 0

        # training
        # set up training mode 
        model.train()

        for itr, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            pred = model(images)

            loss = criterion(pred, labels)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

            # Print/Append activation of the hidden layer 
            # print(model.layer_activations.shape)
            # model.layer_activations

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (images, labels) in enumerate(test_dataloader):
            images, labels = images.to(device), labels.to(device)

            pred = model(images)

            loss = criterion(pred, labels)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if labels[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            hidden_layer_activation_list.append(model.layer_activations)

            
            for activation, label in zip(model.layer_activations, labels):
                # shape of activation and label: 256 and 1 
                
                # get the actual value of item. This is because label is now Tensor 
                label = label.item()

                # this is not part of gradient calculcation 
                with torch.no_grad():
                    activation = activation.numpy()

                # for each image/label, append activation value of neuron 
                for i in range(256):    # number of neurons in hidden layer 
                    _hidden_layer_each_neuron_[i][label].append(activation[i])

        avg_selectivity, std_selectivity = selectivity(_hidden_layer_each_neuron_)
        
        final_selectivity_avg_list.append(avg_selectivity)
        final_selectivity_std_list.append(std_selectivity)

        final_spareness.append(hidden_layer_activation_list)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

    sparsity_list = sparsity_calculator(final_spareness)

    average_sparsity = list()
    for i in range(no_epochs):
        average_sparsity.append( (sparsity_list[i].item()) / 1 )
    # ***************** sparsity calculation ***************** #

    print("average_sparsity:", average_sparsity)

    return test_acc, average_sparsity, final_selectivity_avg_list, final_selectivity_std_list

# Seed 1

In [11]:
torch.manual_seed(1)
np.random.seed(1)

# Adadelta
model_Adadelta = Model()
print("model_Adadelta:", model_Adadelta)
model_Adadelta.to(device)
model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
Adadelta_test_acc, sparsity, Adadelta_avg_selectivity_list, Adadelta_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)

f = open("seed1_sorted_batch_50_Adadelta.txt", "w")
f.write(str(Adadelta_test_acc)+'\n'+str(sparsity)+'\n'+str(Adadelta_avg_selectivity_list)+'\n'+str(Adadelta_std_selectivity_list)+'\n\n')
f.close()

!cp seed1_sorted_batch_50_Adadelta.txt /content/drive/MyDrive


# Adagrad
model_Adagrad = Model()
print("model_Adagrad:", model_Adagrad)
model_Adagrad.to(device)
model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
Adagrad_test_acc, sparsity, Adagrad_avg_selectivity_list, Adagrad_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)

f = open("seed1_sorted_batch_50_Adagrad.txt", "w")
f.write(str(Adagrad_test_acc)+'\n'+str(sparsity)+'\n'+str(Adagrad_avg_selectivity_list)+'\n'+str(Adagrad_std_selectivity_list)+'\n\n')
f.close()

!cp seed1_sorted_batch_50_Adagrad.txt /content/drive/MyDrive


# SGD 
model_SGD = Model()
print("model_SGD:", model_SGD)
model_SGD.to(device)
model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
SGD_test_acc, sparsity, SGD_avg_selectivity_list, SGD_std_selectivity_list = selectivity_trainer(optimizer=optimizer_SGD, model=model_SGD)

f = open("seed1_sorted_batch_50_SGD.txt", "w")
f.write(str(SGD_test_acc)+'\n'+str(sparsity)+'\n'+str(SGD_avg_selectivity_list)+'\n'+str(SGD_std_selectivity_list)+'\n\n')
f.close()

!cp seed1_sorted_batch_50_SGD.txt /content/drive/MyDrive


# Adam 
model_Adam = Model()
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, sparsity, Adam_avg_selectivity_list, Adam_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam)

f = open("seed1_sorted_batch_50_Adam.txt", "w")
f.write(str(Adam_test_acc)+'\n'+str(sparsity)+'\n'+str(Adam_avg_selectivity_list)+'\n'+str(Adam_std_selectivity_list)+'\n\n')
f.close()

!cp seed1_sorted_batch_50_Adam.txt /content/drive/MyDrive

model_Adadelta: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.33790937, Test Loss: 6.81594052, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.22456002, Test Loss: 5.43881819, Test Accuracy: 0.10110000

Epoch: 3/100, Train Loss: 0.13474283, Test Loss: 5.08087767, Test Accuracy: 0.10190000

Epoch: 4/100, Train Loss: 0.11875450, Test Loss: 4.98107031, Test Accuracy: 0.10330000

Epoch: 5/100, Train Loss: 0.11267328, Test Loss: 4.97195589, Test Accuracy: 0.10400000

Epoch: 6/100, Train Loss: 0.11009915, Test Loss: 4.99961284, Test Accuracy: 0.10420000

Epoch: 7/100, Train Loss: 0.10931596, Test Loss: 5.05144711, Test Accuracy: 0.10400000

Epoch: 8/100, Train Loss: 0.10927144, Test Loss: 5.12864606, Test Accuracy: 0.10390000

Epoch: 9/100, Train Loss: 0.10955361, Test Loss: 5.22059197, Test Accuracy: 0.10340000

Epoch: 10/100, Train Loss:

# Seed 100

In [12]:
torch.manual_seed(100)
np.random.seed(100)

# Adadelta
model_Adadelta = Model()
print("model_Adadelta:", model_Adadelta)
model_Adadelta.to(device)
model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
Adadelta_test_acc, sparsity, Adadelta_avg_selectivity_list, Adadelta_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)

f = open("seed100_sorted_batch_50_Adadelta.txt", "w")
f.write(str(Adadelta_test_acc)+'\n'+str(sparsity)+'\n'+str(Adadelta_avg_selectivity_list)+'\n'+str(Adadelta_std_selectivity_list)+'\n\n')
f.close()

!cp seed100_sorted_batch_50_Adadelta.txt /content/drive/MyDrive


# Adagrad
model_Adagrad = Model()
print("model_Adagrad:", model_Adagrad)
model_Adagrad.to(device)
model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
Adagrad_test_acc, sparsity, Adagrad_avg_selectivity_list, Adagrad_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)

f = open("seed100_sorted_batch_50_Adagrad.txt", "w")
f.write(str(Adagrad_test_acc)+'\n'+str(sparsity)+'\n'+str(Adagrad_avg_selectivity_list)+'\n'+str(Adagrad_std_selectivity_list)+'\n\n')
f.close()

!cp seed100_sorted_batch_50_Adagrad.txt /content/drive/MyDrive


# SGD 
model_SGD = Model()
print("model_SGD:", model_SGD)
model_SGD.to(device)
model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
SGD_test_acc, sparsity, SGD_avg_selectivity_list, SGD_std_selectivity_list = selectivity_trainer(optimizer=optimizer_SGD, model=model_SGD)

f = open("seed100_sorted_batch_50_SGD.txt", "w")
f.write(str(SGD_test_acc)+'\n'+str(sparsity)+'\n'+str(SGD_avg_selectivity_list)+'\n'+str(SGD_std_selectivity_list)+'\n\n')
f.close()

!cp seed100_sorted_batch_50_SGD.txt /content/drive/MyDrive


# Adam 
model_Adam = Model()
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, sparsity, Adam_avg_selectivity_list, Adam_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam)

f = open("seed100_sorted_batch_50_Adam.txt", "w")
f.write(str(Adam_test_acc)+'\n'+str(sparsity)+'\n'+str(Adam_avg_selectivity_list)+'\n'+str(Adam_std_selectivity_list)+'\n\n')
f.close()

!cp seed100_sorted_batch_50_Adam.txt /content/drive/MyDrive

model_Adadelta: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.34253885, Test Loss: 6.66815546, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.22628764, Test Loss: 5.36613784, Test Accuracy: 0.10140000

Epoch: 3/100, Train Loss: 0.13400766, Test Loss: 5.02574993, Test Accuracy: 0.10270000

Epoch: 4/100, Train Loss: 0.11760036, Test Loss: 4.93105430, Test Accuracy: 0.10390000

Epoch: 5/100, Train Loss: 0.11161971, Test Loss: 4.92887002, Test Accuracy: 0.10410000

Epoch: 6/100, Train Loss: 0.10908001, Test Loss: 4.97523542, Test Accuracy: 0.10440000

Epoch: 7/100, Train Loss: 0.10863417, Test Loss: 5.05306170, Test Accuracy: 0.10470000

Epoch: 8/100, Train Loss: 0.10906262, Test Loss: 5.11528571, Test Accuracy: 0.10480000

Epoch: 9/100, Train Loss: 0.10971677, Test Loss: 5.19222917, Test Accuracy: 0.10520000

Epoch: 10/100, Train Loss:

# Seed 1234

In [13]:
torch.manual_seed(1234)
np.random.seed(1234)

# Adadelta
model_Adadelta = Model()
print("model_Adadelta:", model_Adadelta)
model_Adadelta.to(device)
model_Adadelta.sigmoid.register_forward_hook(get_activation(model_Adadelta))
optimizer_Adadelta = torch.optim.Adadelta(model_Adadelta.parameters(), lr=1.0)
Adadelta_test_acc, sparsity, Adadelta_avg_selectivity_list, Adadelta_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adadelta, model=model_Adadelta)

f = open("seed1234_sorted_batch_50_Adadelta.txt", "w")
f.write(str(Adadelta_test_acc)+'\n'+str(sparsity)+'\n'+str(Adadelta_avg_selectivity_list)+'\n'+str(Adadelta_std_selectivity_list)+'\n\n')
f.close()

!cp seed1234_sorted_batch_50_Adadelta.txt /content/drive/MyDrive


# Adagrad
model_Adagrad = Model()
print("model_Adagrad:", model_Adagrad)
model_Adagrad.to(device)
model_Adagrad.sigmoid.register_forward_hook(get_activation(model_Adagrad))
optimizer_Adagrad = torch.optim.Adagrad(model_Adagrad.parameters(), lr=0.1)
Adagrad_test_acc, sparsity, Adagrad_avg_selectivity_list, Adagrad_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adagrad, model=model_Adagrad)

f = open("seed1234_sorted_batch_50_Adagrad.txt", "w")
f.write(str(Adagrad_test_acc)+'\n'+str(sparsity)+'\n'+str(Adagrad_avg_selectivity_list)+'\n'+str(Adagrad_std_selectivity_list)+'\n\n')
f.close()

!cp seed1234_sorted_batch_50_Adagrad.txt /content/drive/MyDrive


# SGD 
model_SGD = Model()
print("model_SGD:", model_SGD)
model_SGD.to(device)
model_SGD.sigmoid.register_forward_hook(get_activation(model_SGD))
optimizer_SGD = torch.optim.SGD(model_SGD.parameters(), lr=0.1)
SGD_test_acc, sparsity, SGD_avg_selectivity_list, SGD_std_selectivity_list = selectivity_trainer(optimizer=optimizer_SGD, model=model_SGD)

f = open("seed1234_sorted_batch_50_SGD.txt", "w")
f.write(str(SGD_test_acc)+'\n'+str(sparsity)+'\n'+str(SGD_avg_selectivity_list)+'\n'+str(SGD_std_selectivity_list)+'\n\n')
f.close()

!cp seed1234_sorted_batch_50_SGD.txt /content/drive/MyDrive


# Adam 
model_Adam = Model()
print("model_Adam:", model_Adam)
model_Adam.to(device)
model_Adam.sigmoid.register_forward_hook(get_activation(model_Adam))
optimizer_Adam = torch.optim.Adam(model_Adam.parameters(), lr=0.001)
Adam_test_acc, sparsity, Adam_avg_selectivity_list, Adam_std_selectivity_list = selectivity_trainer(optimizer=optimizer_Adam, model=model_Adam)

f = open("seed1234_sorted_batch_50_Adam.txt", "w")
f.write(str(Adam_test_acc)+'\n'+str(sparsity)+'\n'+str(Adam_avg_selectivity_list)+'\n'+str(Adam_std_selectivity_list)+'\n\n')
f.close()

!cp seed1234_sorted_batch_50_Adam.txt /content/drive/MyDrive

model_Adadelta: Model(
  (linear_1): Linear(in_features=784, out_features=256, bias=True)
  (linear_2): Linear(in_features=256, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Epoch: 1/100, Train Loss: 0.33591804, Test Loss: 6.77225713, Test Accuracy: 0.10090000

Epoch: 2/100, Train Loss: 0.22175279, Test Loss: 5.33478603, Test Accuracy: 0.10100000

Epoch: 3/100, Train Loss: 0.13385248, Test Loss: 4.95120373, Test Accuracy: 0.10220000

Epoch: 4/100, Train Loss: 0.11782450, Test Loss: 4.83367791, Test Accuracy: 0.10380000

Epoch: 5/100, Train Loss: 0.11225529, Test Loss: 4.82234744, Test Accuracy: 0.10460000

Epoch: 6/100, Train Loss: 0.11045434, Test Loss: 4.86903218, Test Accuracy: 0.10470000

Epoch: 7/100, Train Loss: 0.11021173, Test Loss: 4.95318904, Test Accuracy: 0.10440000

Epoch: 8/100, Train Loss: 0.11140450, Test Loss: 5.05721367, Test Accuracy: 0.10390000

Epoch: 9/100, Train Loss: 0.11243132, Test Loss: 5.16634407, Test Accuracy: 0.10340000

Epoch: 10/100, Train Loss: